In [21]:
import sys
sys.path.append('/home/tbrownex/repos/Investing')

In [22]:
import pandas as pd
import numpy_financial as npf
import numpy as np
import datetime

from config import getConfig
from calcIRR import calcIRR

In [23]:
'''def getDays(df):
    startDate = acctValues.iloc[-1]['date']
    startDate = pd.to_datetime(startDate).date()
    endDate = acctValues.iloc[0]['date']
    endDate = pd.to_datetime(endDate).date()

    diff = datetime.date.today()-startDate
    print("{:,} days between {} and {}".format(diff.days, startDate, endDate))
    return diff.days'''

'def getDays(df):\n    startDate = acctValues.iloc[-1][\'date\']\n    startDate = pd.to_datetime(startDate).date()\n    endDate = acctValues.iloc[0][\'date\']\n    endDate = pd.to_datetime(endDate).date()\n\n    diff = datetime.date.today()-startDate\n    print("{:,} days between {} and {}".format(diff.days, startDate, endDate))\n    return diff.days'

In [24]:
cfg = getConfig()
loc = cfg['ETrade']['dataLoc']

In [36]:
transfers = pd.read_csv(loc+'transfers.csv')
acctValues = pd.read_csv(loc+'accountValues.csv')

In [39]:
# Make sure the starting dates are matched between account values and transfers
depStartDt = transfers['date'].min()
acctStartDt = acctValues['date'].min()

startDt = depStartDt if depStartDt > acctStartDt else acctStartDt

transfers = transfers[transfers['date'] >= startDt]
acctValues = acctValues[acctValues['date'] >= startDt]

In [40]:
# The 'initial account value' is basically the first deposit so add a row for it
d = {}
acctValues.sort_values('date', ignore_index=True, inplace=True)
d['amount'] = acctValues.iloc[0]['amount']
d['date'] = acctValues.iloc[0]['date']

transfers.loc[len(transfers)] = d
transfers.sort_values('date', ignore_index=True, inplace=True)

/tmp/ipykernel_1956/1803855359.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transfers.sort_values('date', ignore_index=True, inplace=True)


In [41]:
# Ending value and date
endValue = int(acctValues.iloc[-1]['amount'])
endDate = acctValues.iloc[-1]['date']

endDate = pd.to_datetime(endDate).date()

In [42]:
irr = calcIRR(transfers, endDate, endValue, 100)

Total: 277,084



In [43]:
# Growth of 10k over a year at the computed rate
final = npf.fv(irr, nper=365, pmt=0, pv=-1e4)
print("Annualized rate: {:.2%}".format(final/1e4 -1))

Annualized rate: 8.00%


##### See if it worked

In [32]:
transfers['endDate'] = endDate
transfers['depDate'] = transfers['date'].apply(lambda x: pd.to_datetime(x).date())
transfers['numDays'] = transfers['endDate']-transfers['depDate']
transfers['numDays'] = transfers['numDays'].apply(lambda x: x.days)

In [33]:
def getFV(amt, days):
    return npf.fv(irr, nper=days, pmt=0, pv=-amt)

In [34]:
transfers['FV'] = transfers.apply(lambda row: getFV(row['amount'], row['numDays']), axis=1)
transfers['FV'].sum()

270573.38883840287

In [35]:
transfers

,account,date,amount,endDate,depDate,numDays,FV
0,NaN,2020-12-31,149196,2024-07-31,2020-12-31,1308,196582.090046
1,nonQual,2021-03-15,10000,2024-07-31,2021-03-15,1234,12972.086656
2,nonQual,2021-10-18,5000,2024-07-31,2021-10-18,1017,6195.936586
3,nonQual,2022-04-06,5000,2024-07-31,2022-04-06,847,5977.758250
4,nonQual,2022-05-06,5000,2024-07-31,2022-05-06,817,5940.061543
5,nonQual,2022-06-14,5000,2024-07-31,2022-06-14,778,5891.410992
6,nonQual,2022-08-08,5000,2024-07-31,2022-08-08,723,5823.477681
7,nonQual,2022-11-01,5000,2024-07-31,2022-11-01,638,5720.027372
8,nonQual,2023-02-07,5000,2024-07-31,2023-02-07,540,5603.033994
9,nonQual,2023-07-31,5000,2024-07-31,2023-07-31,366,5401.175889


In [20]:
acctValues

,date,amount
0,2020-12-31,149196
1,2021-01-31,145792
2,2021-02-28,151239
3,2021-03-31,164450
4,2021-04-30,168417
5,2021-05-31,175073
6,2021-06-30,174397
7,2021-07-31,174307
8,2021-08-31,177323
9,2021-09-30,173553
